# EnMAP L2A HSI Products

EnMAP (Environmental Mapping and Analysis Program) is the first German hyperspectral satellite mission for Earth observation. Imaging spectrometers measure solar radiation reflected by Earth’s surface from visible light to short-wave infrared. This allows precise conclusions to be drawn about the condition of and changes to the planet’s surface.

The 'EnMAP L2A HSI Products'-collection provides all 224 bands as [CEOS-ARD](https://ceos.org/ard/) data using athmosperic correction over land.
L2A Products provide a complete atmospheric correction of the orthorectified EnMAP L1C product. The L1C processor orthorectifies image tiles from the VNIR and SWIR instrument independently. After the orthorectification the two image tiles are co-registered (requirement better than 0.2-pixel size) and form a geometric consistent product over the whole wavelength range.
The result of the atmosphere correction is reflectance [in % units] values and pixel classification (quality layers).

One Product consists of the following files:

- ENMAP.HSI.L2A-SPECTRAL_IMAGE
- ENMAP.HSI.L2A-METADATA
- ENMAP.HSI.L2A-STAC_JSON
- ENMAP.HSI.L2A-QL
    - QL_VNIR
    - QL_VNIR_THUMBNAIL
    - QL_SWIR
    - QL_QUALITY_CLASSES
    - QL_QUALITY_CLOUD
    - QL_QUALITY_CLOUDSHADOW
    - QL_QUALITY_HAZE
    - QL_QUALITY_CIRRUS
    - QL_QUALITY_SNOW
    - QL_QUALITY_TESTFLAGS
    - QL_QUALITY_PIXELMASK


This notebook provides informations on how to programmatically access this collection, which is using the UserManagementSystem [UMS] to authenticate users. To execute the script and the terminal commands, a valid useraccount is needed for the requested dataset.



## Download using the STAC-API and python

The first step is to connect to the [EOC Geoservice STAC API](https://geoservice.dlr.de/eoc/ogc/stac/v1) and to query the [EnMAP L2A HSI](https://geoservice.dlr.de/eoc/ogc/stac/v1/collections/ENMAP_HSI_L2A) collection.
In the first case we query data that have the boundingbox (bbox) of the DLR Oberpfaffenhofen and were acquired in the year 2023. 4 Items will be found.
In the second query we have the same bbox and time but add a filter for "enmap:overallQuality" to gain only results with a "nominal quality".

In [8]:
from pystac_client import Client

catalog = Client.open("https://geoservice.dlr.de/eoc/ogc/stac/v1/")
collections = ["ENMAP_HSI_L2A"]
bbox = [11.230259, 48.051808, 11.337891, 48.117059] #DLR Oberpfaffenhofen
dates = ["2023-01-01", "2023-12-31"]
parameters = "enmap:overallQuality=0"


search = catalog.search(
    collections= collections, 
    bbox= bbox,
    datetime=dates
)

items = search.item_collection()
print(f"{len(items)} items found")
items


4 items found


id: ENMAP01-____L2A-DT0000024647_20230629T110132Z_025_V010402_20240630T020748Z
"bbox: [11.0001767, 47.8630601, 11.5537747, 48.2048981]"
version: 01.04.02
created: 2024-06-30T09:24:48.048+00:00
updated: 2024-06-30T09:24:48.048+00:00
datetime: 2023-06-29T11:01:32Z
start_datetime: 2023-06-29T11:01:32.798420Z
end_datetime: 2023-06-29T11:01:37.333108Z
platform: enmap
proj:epsg: None
sat:relative_orbit: 6689


In the second query we have the same bbox and time but add a filter for "enmap:overallQuality" to gain only results with a "nominal quality".

In [10]:
search2 = catalog.search(
    collections= collections, 
    bbox= bbox,
    datetime=dates,
    filter_lang="cql2-text",
    filter=parameters
)

items2 = search2.item_collection()
print(f"{len(items2)} items found")
items2

1 items found


id: ENMAP01-____L2A-DT0000024647_20230629T110132Z_025_V010402_20240630T020748Z
"bbox: [11.0001767, 47.8630601, 11.5537747, 48.2048981]"
version: 01.04.02
created: 2024-06-30T09:24:48.048+00:00
updated: 2024-06-30T09:24:48.048+00:00
datetime: 2023-06-29T11:01:32Z
start_datetime: 2023-06-29T11:01:32.798420Z
end_datetime: 2023-06-29T11:01:37.333108Z
platform: enmap
proj:epsg: None
sat:relative_orbit: 6689


After the query parameters are as desired, we can download the data using the script provided below. 
- In Line 5 the path has to be specified, where the data should be stored. The script will create subfolders for every Item. 
- in Line 8 the EnMAP-Account credentials have to be provided (also possible to save them in a external file and import them, so that they dont have to be provided in plaintext)

In [ ]:
import os
import urllib3
import shutil

path = r"*****location*****"   #Add location, for example /localhome/user_name/projects/enmap for linux

http = urllib3.PoolManager()
header = urllib3.make_headers(basic_auth='***USER***:***PASSWORD***') #Add your User / PW

for item in items:
    download_path = os.path.join(path, item.collection_id, item.id)
    if not os.path.exists(download_path):
        os.makedirs(download_path, exist_ok=True)
        print("Folder {} was created.".format(download_path))
    print("=====================")
    for name, asset in item.assets.items():
        with http.request('GET', asset.href, headers=header, preload_content=False) as a, open(os.path.join(download_path, asset.href.rsplit('/',1)[1]), 'wb') as out_file:
            shutil.copyfileobj(a, out_file)
    print("The following data: ", os.listdir(download_path), "was downloaded to the folder: {}.".format(download_path))
    print("=====================")

## Download using CLI-Tools

To download data using just CLI-Tools one has to first get the session cookie from the UMS. 
The easiest way is to log into the UMS using the Browser. 

In this case we go to the Geoservice [EnMAP L2A Download page](https://download.geoservice.dlr.de/ENMAP/files/L2A/). We will be reconnected to the [UMS](https://sso.eoc.dlr.de/eoc/auth/login?service=https://download.geoservice.dlr.de/ENMAP/files/L2A/) (see Image 1). Before entering the login credentials, open the "Web Developer Tool" (Firefox: F12 or using settings) and switch to the "Network" Context. Now enter the credentials and login. There will be a GET-request (see Image 2) to download.geoservice.dlr.de with the file name that was accessed, this request contains the Session-Cookie that we need. We can just right click on the GET-request and under "Copy Value" choose "Copy as cURL". With this copied cURL-command we can just run the same download again, but from the command line. To start a additional download, just exchange the URL that is used in the command.  

Here an example of a cURL-request, to make it work, the ***=====COOKIE=====*** has to be adapted.

### cURL

curl 'https://download.geoservice.dlr.de/ENMAP/files/L2A/2024/05/10/DT0000073114/05/ENMAP01-____L2A-DT0000073114_20240510T211450Z_005_V010402_20240523T183506Z-METADATA.XML' -H 'User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8' -H 'Accept-Language: en-US,en;q=0.5' -H 'Accept-Encoding: gzip, deflate, br' -H 'DNT: 1' -H 'Connection: keep-alive' -H 'Cookie: session=***=====COOKIE=====***' -H 'Upgrade-Insecure-Requests: 1' -H 'Sec-Fetch-Dest: document' -H 'Sec-Fetch-Mode: navigate' -H 'Sec-Fetch-Site: same-origin' -H 'Sec-Fetch-User: ?1'

### wget

To use the same command with wget we just have to interchange the -H parameters to --header parameters. 

wget 'https://download.geoservice.dlr.de/ENMAP/files/L2A/2024/05/12/DT0000072933/01/ENMAP01-____L2A-DT0000072933_20240512T115339Z_001_V010402_20240515T224006Z-SPECTRAL_IMAGE_COG.TIF' --header 'User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Accept-Encoding: gzip, deflate, br' --header 'DNT: 1' --header 'Connection: keep-alive' --header 'Cookie: session=***=====COOKIE=====***' --header 'Upgrade-Insecure-Requests: 1' --header 'Sec-Fetch-Dest: document' --header 'Sec-Fetch-Mode: navigate' --header 'Sec-Fetch-Site: same-site' --header 'Sec-Fetch-User: ?1'









![cli_2.png](cli_2.png)
*Image 1: UMS Login Page*



![cli_1.png](cli_1.png)
*Image 2: Session Cookies in the "Web Developer Tool"